# Building a Financial Analyst

## Setup

### Install Packages

In [1]:
#Need to install Docker also
#Check with docker --version on terminal/command line to check if Docker is installed
# export PATH="$PATH:/Applications/Docker.app/Contents/Resources/bin/"

In [2]:
## Necessary as crewai-tools==0.71.0 required and uv is not able to resolve dependencies
# !uv pip install --upgrade crewai-tools

Resolve this issue in CrewAI before proceeding: https://github.com/crewAIInc/crewAI/issues/2606

### API Keys

In [3]:
import os, json, re, getpass
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [4]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

In [5]:
#Check for Groq API Key
_set_env("GROQ_API_KEY")

### Libraries

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
import re
import json
import os
import yfinance as yf
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import CodeInterpreterTool, FileReadTool
from pydantic import BaseModel, Field
from IPython.display import display, Markdown

## Selecting Components

### Chat Model

In [8]:
llm = LLM(
    # model="groq/llama-3.1-8b-instant",
    model="openai/gpt-4o-mini",
    # temperature=0.5,
    # max_completion_tokens=1024,
    # top_p=0.9,
    # stop=None,
    # stream=False,
)

### Structured Output for Query Analysis

**NOTE:**: When defining state in LangGraph, you have two primary options: **TypedDict** and **Pydantic BaseModel**. The best choice depends on your specific needs:

**TypedDict** is a good fit if you're looking for a *lightweight solution without strict runtime type enforcement*. It's simple to use and ideal for smaller applications where the overhead of a more robust system isn't necessary.

For *greater robustness and early error detection*, especially in larger or more complex applications, **Pydantic BaseModel** is the superior choice. Pydantic provides powerful data validation and parsing, catching type-related errors early in the development process and improving the overall reliability of your application.

In [9]:
class QueryAnalysisOutput(BaseModel):
    """Structured output for the query analysis task."""
    symbol: str = Field(..., description="Stock ticker symbol (e.g., TSLA, AAPL).")
    timeframe: str = Field(..., description="Time period (e.g., '1d', '1mo', '1y').")
    action: str = Field(..., description="Action to be performed (e.g., 'fetch', 'plot').")

## Setting up Crew of Agents

In CrewAI, you define **agents** as independent workers, each with a specific job and objective. You then assign them **tasks**, which are detailed instructions on what they need to do to reach that objective. You can also assign **tools** directly as tasks for agents to use.

**Let's first create our CrewAI agents with role, main goal/objective, and backstory/personality.**

### Query Parser Agent

In [10]:
query_parser_agent = Agent(
    role="Stock Data Analyst",
    goal="Extract stock details and fetch required data from this user query: {query}.",
    backstory="You are a financial analyst specializing in stock market data retrieval.",
    llm=llm,
    verbose=True,
    memory=True,
)

### Code Writer Agent

In [11]:
code_writer_agent = Agent(
    role="Senior Python Developer",
    goal="Write Python code to visualize stock data.",
    backstory="""You are a Senior Python developer specializing in stock market data visualization. 
                 You are also a Pandas, Matplotlib and yfinance library expert.
                 You are skilled at writing production-ready Python code""",
    llm=llm,
    verbose=True,
)

### Code Interpreter Agent

**Note:** By default, the Code Interpreter Tool is prompted by CrewAI to run on Docker to safeguard against any usnafe action that the code can take, since it'll run in a container (i.e., `code_interpreter_tool = CodeInterpreterTool()`  or `allow_code_execution=True`).

To bypass this, we can set `code_interpreter_tool = CodeInterpreterTool(unsafe_mode=True)` and use this only when defining the agent (not `allow_code_execution=True`) 

In [12]:
# code_interpreter_tool = CodeInterpreterTool(unsafe_mode=True)

code_execution_agent = Agent(
    role="Senior Code Execution Expert",
    goal="Review and execute the generated Python code by code writer agent to visualize stock data.",
    backstory="You are a code execution expert. You are skilled at executing Python code.",
    # tools=[code_interpreter_tool],
    allow_code_execution=True,   # This automatically adds the CodeInterpreterTool
    llm=llm,
    verbose=True,
)

**Now, let's define the agents' tasks.**

### Query Parser Agent - Task

In [13]:
query_parsing_task = Task(
    description="Analyze the user query and extract stock details.",
    expected_output="A dictionary with keys: 'symbol', 'timeframe', 'action'.",
    output_pydantic=QueryAnalysisOutput,
    agent=query_parser_agent,
)

### Code Writer Agent - Task

In [14]:
code_writer_task = Task(
    description="""Write Python code to visualize stock data based on the inputs from the stock analyst
                   where you would find stock symbol, timeframe and action.""",
    expected_output="A clean and executable Python script file (.py) for stock visualization.",
    agent=code_writer_agent,
)

### Code Interpreter Agent - Task

In [15]:
code_execution_task = Task(
    description="""Review and execute the generated Python code by code writer agent to visualize stock data.""",
    expected_output="A clean and executable Python script file (.py) for stock visualization.",
    agent=code_execution_agent,
)

**Now, let's create crew to manage agents and task workflow.**

In [16]:
crew = Crew(
    agents=[query_parser_agent, code_writer_agent, code_execution_agent],
    tasks=[query_parsing_task, code_writer_task, code_execution_task],
    process=Process.sequential
)

In [17]:
# Run the crew with an example query
result = crew.kickoff(inputs={"query": "Plot YTD stock gain of Tesla"})

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Stock Data Analyst                                                                                      │
│                                                                                                                 │
│  Task: Analyze the user query and extract stock details.                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Stock Data Analyst                                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "symbol": "TSLA",                                                                                            │
│    "timeframe": "YTD",                                                                                          │
│    "action": "Plot stock gain"                                                                                  │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Python Developer                                                                                 │
│                                                                                                                 │
│  Task: Write Python code to visualize stock data based on the inputs from the stock analyst                     │
│                     where you would find stock symbol, timeframe and action.                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Python Developer                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```python                                                                                                      │
│  import yfinance as yf                                                                                          │
│  import matplotlib.pyplot as plt                                                                                │
│  import pandas as pd                                                                                            │
│  import datetime                                                                                                │
│                                                                                                                 │
│  def plot_stock_gain(symbol, timeframe):                                                                        │
│      # Fetch the stock data using yfinance                                                                      │
│      stock_data = yf.Ticker(symbol)                                                                             │
│                                                                                                                 │
│      # Define the start date based on the timeframe                                                             │
│      if timeframe == "YTD":                                                                                     │
│          start_date = datetime.datetime(datetime.datetime.now().year, 1, 1)                                     │
│      else:                                                                                                      │
│          raise ValueError("Unsupported timeframe provided. Please use 'YTD'.")                                  │
│                                                                                                                 │
│      # Get historical market data                                                                               │
│      hist = stock_data.history(start=start_date)                                                                │
│                                                                                                                 │
│      # Calculate the stock gain                                                                                 │
│      hist['Gain'] = hist['Close'].pct_change() * 100  # Percentage gain                                         │
│                                                                                                                 │
│      # Plotting the stock gain                                                                                  │
│      plt.figure(figsize=(12, 6))                                                                                │
│      plt.plot(hist.index, hist['Gain'], label='Daily Gain (%)', color='blue', linewidth=2)                      │
│                                                                                                                 │
│      # Formatting the plot                                                                                      │
│      plt.title(f"{symbol} Daily Gain {timeframe}", fontsize=16)                                                 │
│      plt.xlabel("Date", fontsize=12)                                                                            │
│      plt.ylabel("Gain (%)", fontsize=12)               

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Code Execution Expert                                                                            │
│                                                                                                                 │
│  Task: Review and execute the generated Python code by code writer agent to visualize stock data.               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│                                   Open        High  ...  Dividends  Stock Splits                                │
│  Date                                               ...                                                         │
│  2025-01-02 00:00:00-05:00  390.100006  392.730011  ...        0.0           0.0                                │
│  2025-01-03 00:00:00-05:00  381.480011  411.880005  ...        0.0           0.0                                │
│  2025-01-06 00:00:00-05:00  423.200012  426.429993  ...        0.0           0.0                                │
│  2025-01-07 00:00:00-05:00  405.829987  414.329987  ...        0.0           0.0                                │
│  2025-01-08 00:00:00-05:00  392.950012  402.500000  ...        0.0           0.0                                │
│  ...                               ...         ...  ...        ...           ...                                │
│  2025-09-05 00:00:00-04:00  348.000000  355.869995  ...        0.0           0.0                                │
│  2025-09-08 00:00:00-04:00  354.640015  358.440002  ...        0.0           0.0                                │
│  2025-09-09 00:00:00-04:00  348.440002  350.769989  ...        0.0           0.0                                │
│  2025-09-10 00:00:00-04:00  350.549988  356.329987  ...        0.0           0.0                                │
│  2025-09-11 00:00:00-04:00  350.170013  368.989990  ...        0.0           0.0                                │
│                                                                                                                 │
│  [173 rows x 7 columns]                                                                                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Code Execution Expert                                                                            │
│                                                                                                                 │
│  Thought: Thought: I have successfully fetched the stock data for TSLA but the full visualization code could    │
│  not be executed due to a missing library. I will now attempt to run the full code again after ensuring all     │
│  required libraries are present.                                                                                │
│                                                                                                                 │
│  Using Tool: Code Interpreter                                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"code\": \"import yfinance as yf\\nimport matplotlib.pyplot as plt\\nimport pandas as pd\\nimport           │
│  datetime\\n\\ndef plot_stock_gain(symbol, timeframe):\\n    # Fetch the stock data using yfinance\\n           │
│  stock_data = yf.Ticker(symbol)\\n    \\n    # Define the start date based on the timeframe\\n    if timeframe  │
│  == \\\"YTD\\\":\\n        start_date = datetime.datetime(datetime.datetime.now().year, 1, 1)\\n    else:\\n    │
│  raise ValueError(\\\"Unsupported timeframe provided. Please use 'YTD'.\\\")\\n\\n    # Get historical market   │
│  data\\n    hist = stock_data.history(start=start_date)\\n\\n    # Calculate the stock gain\\n    hist['Gain']  │
│  = hist['Close'].pct_change() * 100  # Percentage gain\\n\\n    # Plotting the stock gain\\n                    │
│  plt.figure(figsize=(12, 6))\\n    plt.plot(hist.index, hist['Gain'], label='Daily Gain (%)', color='blue',     │
│  linewidth=2)\\n    \\n    # Formatting the plot\\n    plt.title(f\\\"{symbol} Daily Gain {timeframe}\\\",      │
│  fontsize=16)\\n    plt.xlabel(\\\"Date\\\", fontsize=12)\\n    plt.ylabel(\\\"Gain (%)\\\", fontsize=12)\\n    │
│  plt.axhline(0, color='red', linestyle='--', linewidth=1)\\n    plt.legend()\\n    plt.grid()\\n\\n    # Show   │
│  the plot\\n    plt.tight_layout()\\n    plt.show()\\n\\n# Example usage\\nif __name__ == \\\"__main__\\\":\\n  │
│  symbol = \\\"TSLA\\\"\\n    timeframe = \\\"YTD\\\"\\n    plot_stock_gain(symbol, timeframe)\",                │
│  \"libraries_used\": [\"yfinance\", \"matplotlib\", \"pandas\"]}"                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Something went wrong while running the code:                                                                   │
│  Traceback (most recent call last):                                                                             │
│    File "<string>", line 2, in <module>                                                                         │
│  ModuleNotFoundError: No module named 'matplotlib'                                                              │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:          │
│                                                                                                                 │
│  Tool Name: Code Interpreter                                                                                    │
│  Tool Arguments: {'code': {'description': 'Python3 code used to be interpreted in the Docker container. ALWAYS  │
│  PRINT the final result and the output of the code', 'type': 'str'}, 'libraries_used': {'description': 'List    │
│  of libraries used in the code with proper installing names separated by commas. Example:                       │
│  numpy,pandas,beautifulsoup4', 'type': 'list[str]'}}                                                            │
│  Tool Description: Interprets Python3 code strings with a final print statement.                                │
│                                                                                                                 │
│  IMPORTANT: Use the following format in your response:                                                          │
│                                                                                                                 │
│  ```                                                                                                            │
│  Thought: you should always think about what to do                                                              │
│  Action: the action to take, only one name of [Code Interpreter], just the name, exactly as it's written.       │
│  Action Input: the input to the action, just a simple JSON object, enclosed in curly braces, using " to wrap    │
│  keys and values.                                                                                               │
│  Observation: the result of the action                                                                          │
│  ```                                                                                                            │
│                                                                                                                 │
│  Once all necessary information is gathered, return the following format:                                       │
│                                                                                                                 │
│  ```                                                                                                            │
│  Thought: I now know the final answer                                                                           │
│  Final Answer: the final answer to the original input question                                                  │
│  ```                                                                                                            │
│                                                       

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Code Execution Expert                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  import yfinance as yf                                                                                          │
│  import matplotlib.pyplot as plt                                                                                │
│  import pandas as pd                                                                                            │
│  import datetime                                                                                                │
│                                                                                                                 │
│  def plot_stock_gain(symbol, timeframe):                                                                        │
│      # Fetch the stock data using yfinance                                                                      │
│      stock_data = yf.Ticker(symbol)                                                                             │
│                                                                                                                 │
│      # Define the start date based on the timeframe                                                             │
│      if timeframe == "YTD":                                                                                     │
│          start_date = datetime.datetime(datetime.datetime.now().year, 1, 1)                                     │
│      else:                                                                                                      │
│          raise ValueError("Unsupported timeframe provided. Please use 'YTD'.")                                  │
│                                                                                                                 │
│      # Get historical market data                                                                               │
│      hist = stock_data.history(start=start_date)                                                                │
│                                                                                                                 │
│      # Calculate the stock gain                                                                                 │
│      hist['Gain'] = hist['Close'].pct_change() * 100  # Percentage gain                                         │
│                                                                                                                 │
│      # Plotting the stock gain                                                                                  │
│      plt.figure(figsize=(12, 6))                                                                                │
│      plt.plot(hist.index, hist['Gain'], label='Daily Gain (%)', color='blue', linewidth=2)                      │
│                                                                                                                 │
│      # Formatting the plot                                                                                      │
│      plt.title(f"{symbol} Daily Gain {timeframe}", fontsize=16)                                                 │
│      plt.xlabel("Date", fontsize=12)                                                                            │
│      plt.ylabel("Gain (%)", fontsize=12)                                                                        │
│      plt.axhline(0, color='red', linestyle='--', linewi

In [18]:
#Print result
print(result.raw)

import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import datetime

def plot_stock_gain(symbol, timeframe):
    # Fetch the stock data using yfinance
    stock_data = yf.Ticker(symbol)
    
    # Define the start date based on the timeframe
    if timeframe == "YTD":
        start_date = datetime.datetime(datetime.datetime.now().year, 1, 1)
    else:
        raise ValueError("Unsupported timeframe provided. Please use 'YTD'.")

    # Get historical market data
    hist = stock_data.history(start=start_date)

    # Calculate the stock gain
    hist['Gain'] = hist['Close'].pct_change() * 100  # Percentage gain

    # Plotting the stock gain
    plt.figure(figsize=(12, 6))
    plt.plot(hist.index, hist['Gain'], label='Daily Gain (%)', color='blue', linewidth=2)
    
    # Formatting the plot
    plt.title(f"{symbol} Daily Gain {timeframe}", fontsize=16)
    plt.xlabel("Date", fontsize=12)
    plt.ylabel("Gain (%)", fontsize=12)
    plt.axhline(0, color='red', linest